In [ ]:
import scipp as sc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
# Load a single vanadium data file and plot it

filename = '../../IN16b_GGG_data/vanadium_Q8.dat'

data_array = np.loadtxt(filename)
EnergyValues=data_array[:, 0]
IntensityValues=data_array[:,1]
ErrorValues=data_array[:,2]

Energy=sc.array(dims=['Energy'],values=EnergyValues/1000,unit='meV')
Intensity=sc.array(dims=['Energy'],values=IntensityValues,variances=ErrorValues*ErrorValues) #The variance is the square of the uncertainty!

VanadiumData = sc.DataArray(data=Intensity, coords={'Energy': Energy})


IntensityMin=0.0
IntensityMax=0.06

start = -0.02 * sc.Unit('meV')
stop = 0.02 * sc.Unit('meV')
sc.plot(VanadiumData['Energy',start:stop],vmin=IntensityMin,vmax=IntensityMax)



In [ ]:
# Play a bit around with plot options
IntensityMin=0.0
IntensityMax=0.06
binnedVanadiumData = VanadiumData.bin(Energy=sc.scalar(1e-3*0.1, unit='meV')) 
sc.plot(binnedVanadiumData['Energy',start:stop].bins.mean(),vmin=IntensityMin,vmax=IntensityMax,marker='o',markerfacecolor='none',color='black',linestyle='none')



In [40]:
from easyCore.Objects.Variable import Parameter
from easyCore.Objects.ObjectClasses import BaseObj
from easyCore.Fitting.Fitting import Fitter


#Assume just a Gaussian for now

#TODO: Would be neat to do this with units
energy_offset_guess=-0.5*1e-3
res_gauss1_area_guess=0.01
res_gauss1_sigma_guess=0.3*1e-3
res_BG_guess=1e-4

energy_offset    =Parameter(name='energy_offset',   value=energy_offset_guess   , fixed=False)
res_gauss1_area  =Parameter(name='res_gauss1_area', value=res_gauss1_area_guess , fixed=False)
res_gauss1_sigma =Parameter(name='res_gauss1_sigma',value=res_gauss1_sigma_guess, fixed=False)
res_BG           =Parameter(name='res_BG',          value=res_BG_guess,fixed=False)

# TODO: What is up with this way of doing things; the function takes parameters from outside the function???
# TODO: How to use functions from QENSlibrary?
def resolution_function(x: np.ndarray) -> np.ndarray:
    """
    (Gaussian) resolution function
    
    :x: values to calculate the model over. 
    
    :return: model values.
    """

    x=x-energy_offset.raw_value

    y=res_gauss1_area.raw_value*1/np.sqrt(2*np.pi)/res_gauss1_sigma.raw_value*np.exp(-0.5*(x/res_gauss1_sigma.raw_value)**2  )+res_BG.raw_value

    return y

# def resolution_function(x: np.ndarray,p:np.array) -> np.ndarray:
#     """
#     (Gaussian) resolution function
    
#     :x: values to calculate the model over. 
#     :p[0]: Area of Gaussian
#     :p[1]: Center of Gaussian
#     :p[2]: Sigma of Gaussian
#     :p[3]: Flat background
    
#     :return: model values.
#     """
#     res_gauss1_area=p[0]
#     energy_offset=p[1]
#     res_gauss1_sigma=p[2]
#     res_BG=p[3]

#     x=x-energy_offset.raw_value

#     y=res_gauss1_area*1/np.sqrt(2*np.pi)/res_gauss1_sigma*np.exp(-0.5*(x/res_gauss1_sigma)**2  )+res_BG

#     return y

# p=[res_gauss1_area, energy_offset,res_gauss1_sigma,res_BG]
resolution = BaseObj(name='resolution', energy_offset=energy_offset, res_gauss1_area=res_gauss1_area, res_gauss1_sigma=res_gauss1_sigma)
f = Fitter(resolution, resolution_function)

# Extract coordinates
energy_to_be_fitted   =        sc.midpoints(binnedVanadiumData['Energy',start:stop].coords.get('Energy')).values
# energy_to_be_fitted   = (energy_to_be_fitted[1]-energy_to_be_fitted[0])/2+energy_to_be_fitted[:-1]

intensity_to_be_fitted=        binnedVanadiumData['Energy',start:stop].bins.mean().values
error_to_be_fitted    =np.sqrt(binnedVanadiumData['Energy',start:stop].bins.mean().variances)

fit_result = f.fit(x=energy_to_be_fitted, y=intensity_to_be_fitted, weights=1/error_to_be_fitted)

# Save the resolution parameters, because of the global variable issue

res_gauss1_area_fit=res_gauss1_area
res_gauss1_sigma_fit=res_gauss1_sigma



In [41]:
print(energy_offset)
print(res_gauss1_area)
print(res_gauss1_sigma)
print(res_BG)
print(resolution)
# y_test=resolution_function(EnergyValues)
# print(y_test[500])
# print(len(y_test))
# print(res_gauss1_area.raw_value)
# print(EnergyValues)

<Parameter 'energy_offset': -0.000291+/-0.000010, bounds=[-inf:inf]>
<Parameter 'res_gauss1_area': (5.44+/-0.26)e-05, bounds=[-inf:inf]>
<Parameter 'res_gauss1_sigma': 0.000429+/-0.000006, bounds=[-inf:inf]>
<Parameter 'res_BG': 0.0001+/-0, bounds=[-inf:inf]>
BaseObj `resolution`


In [ ]:
#TODO: Figure out how to make the x axis of the VanadiumFit different from VanadiumData. Consider of VanadiumData can/should contain everything
y2=sc.array(dims=['Energy'],values=resolution_function(VanadiumData.coords.get('Energy').values)) 

VanadiumFit = sc.DataArray(data=y2, coords={'Energy': Energy})
# sc.plot(VanadiumFit)

In [ ]:
# Plot data and fit
# Play a bit around with plot options
IntensityMin=0.0
IntensityMax=0.06
sc.plot({'Data': binnedVanadiumData['Energy',start:stop].bins.mean(),
         'Fit': VanadiumFit},
        vmin=IntensityMin,vmax=IntensityMax,marker={'Data':'o','Fit': 'none'},markerfacecolor='none',color={'Data': 'black', 'Fit': 'red'},linestyle={'Data': 'none','Fit': '-'})

# sc.plot(binnedVanadiumData['Energy',start:stop].bins.mean(),vmin=IntensityMin,vmax=IntensityMax,marker='o',markerfacecolor='none',color='blue',linestyle='--')


# y_fit=resolution_function(energy_to_be_fitted)
# y_fit
# sc.plot(energy_to_be_fitted,y_fit)


In [ ]:

# sc.plot({'Data': binnedVanadiumData['Energy',start:stop].bins.mean(),
#          'Fit': VanadiumFit},
#         vmin=IntensityMin,vmax=IntensityMax,marker={'Data':'o','Fit': 'none'},markerfacecolor='none',color={'Data': 'black', 'Fit': 'red'},linestyle={'Data': 'none','Fit': '-'})

mygroup=sc.DataGroup({'Data': binnedVanadiumData.bins.mean(),
          'Fit': VanadiumFit})

mygroup['Energy',start:stop].plot(linestyle={'Data': 'none','Fit': '-'},color={'Data': 'black', 'Fit': 'red'})



In [ ]:
#TODO: Would be neat to do this with units
# energy_offset_guess=-0.5*1e-3
# res_gauss1_area_guess=0.01
# res_gauss1_sigma_guess=0.3*1e-3
# res_BG_guess=1e-4

# energy_offset    =Parameter(name='energy_offset',   value=energy_offset_guess   , fixed=False)
# res_gauss1_area  =Parameter(name='res_gauss1_area', value=res_gauss1_area_guess , fixed=False)
# res_gauss1_sigma =Parameter(name='res_gauss1_sigma',value=res_gauss1_sigma_guess, fixed=False)
# res_BG           =Parameter(name='res_BG',          value=res_BG_guess,fixed=False)

# TODO: Use functions from QENSlibrary?
def resolution_function_scipp(Energy,energy_offset=0*sc.Unit('meV'),res_gauss1_area=1*sc.Unit('meV'),res_gauss1_sigma=0.1*sc.Unit('meV'),res_BG=0):
    """
    (Gaussian) resolution function
    
    :x: values to calculate the model over. 
    
    :return: model values.
    """
    x=Energy
    x=x-energy_offset

    y=res_gauss1_area*1/sc.sqrt(2*np.pi)/res_gauss1_sigma*sc.exp(-0.5*(x/res_gauss1_sigma)**2  )+res_BG

    return y

from scipp import curve_fit

# popt, _ = curve_fit(['Energy'], resolution_function_scipp, VanadiumData, 
#                     p0 = {'energy_offset': -0.002 * sc.Unit('meV'),
#                           'res_gauss1_area':0.005 /sc.Unit('meV'),
#                           'res_gauss1_sigma':0.005/sc.Unit('meV'),
#                           'res_BG':1e-5})

popt, _ = curve_fit(['Energy'], resolution_function_scipp, VanadiumData)



In [30]:
1/np.sqrt(2*np.pi)

0.3989422804014327

In [75]:
def round(a, d):
    'Helper for the doctests'
    return sc.round(10**d * a) / 10**d

def resolution_function_scipp(Energy, energy_offset,res_gauss1_area, res_gauss1_sigma,res_BG):
    x=Energy
    x=x-energy_offset

    y=res_gauss1_area*1/np.sqrt(2*np.pi)/res_gauss1_sigma*sc.exp(-0.5*(x/res_gauss1_sigma)**2  )+res_BG
    return y


from scipp import curve_fit
popt, _ = curve_fit(['Energy'], resolution_function_scipp, VanadiumData, 
                    p0 = {'res_gauss1_area': 5e-5*sc.Unit('meV'), 
                          'res_gauss1_sigma': 4e-4 * sc.Unit('meV'),
                          'energy_offset': 0 * sc.Unit('meV'),
                          'res_BG':1e-5})
# print(round(sc.values(popt['res_gauss1_area']), 6), round(sc.stddevs(popt['res_gauss1_area']), 6))
# print(round(sc.values(popt['res_gauss1_sigma']), 6), round(sc.stddevs(popt['res_gauss1_sigma']), 6))

# print(sc.values(popt['res_gauss1_area']))
# print(da)
# print(VanadiumData)



y2=sc.array(dims=['Energy'],values=
            resolution_function_scipp(sc.values(VanadiumData.coords['Energy']),
                                      energy_offset=sc.values(popt['energy_offset']),
                                      res_gauss1_area=sc.values(popt['res_gauss1_area']),
                                      res_gauss1_sigma=sc.values(popt['res_gauss1_sigma']),
                                      res_BG=sc.values(popt['res_BG']))) 

VanadiumFit = sc.DataArray(data=y2, coords={'Energy': Energy})





In [73]:
VanadiumFit
# sc.plot(VanadiumFit)

# mygroup=sc.DataGroup({'Data': VanadiumData,
#           'Fit': VanadiumFit})

# mygroup.plot()

#mygroup['Energy',start:stop].plot(linestyle={'Data': 'none','Fit': '-'},color={'Data': 'black', 'Fit': 'red'})



<scipp.DataArray>
Dimensions: Sizes[Energy:1024, ]
Coordinates:
* Energy                    float64            [meV]  (Energy)  [-0.029678, -0.02962, ..., 0.02962, 0.029678]
Data:
                           PyObject        <no unit>  (Energy)  [<scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64  [dimensionless]  ()  0.000123363

, <scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64  [dimensionless]  ()  0.000123363

, ..., <scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64  [dimensionless]  ()  0.000123363

, <scipp.DataArray>
Dimensions: Sizes[]
Data:
                            float64  [dimensionless]  ()  0.000123363

]

In [60]:
# sc.values(popt)
sc.values(VanadiumData.coords['Energy'])
#y2=sc.array(dims=['Energy'],values=resolution_function_scipp(VanadiumData.coords.get('Energy').values,sc.values(popt))) 


<scipp.Variable> (Energy: 1024)    float64            [meV]  [-0.029678, -0.02962, ..., 0.02962, 0.029678]